In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Tesi"

/content/drive/MyDrive/Tesi


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import h5py
import tensorflow as tf 
from tensorflow import keras
import pickle
import numpy as np

import os
import h5py
import math
import tensorflow as tf 
from tensorflow import keras
import pickle
import numpy as np
from sklearn.utils import shuffle
import os
import h5py
import math
from keras.activations import sigmoid
from keras.activations import sigmoid
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from keras.metrics import FalsePositives, TruePositives, FalseNegatives, TrueNegatives

## Train 300

In [ ]:
import tensorflow as tf 
from tensorflow import keras
import pickle
import numpy as np

import os
import h5py
import math

p =  str('20')
patient = "chb" + p

with open("./CHB-Definitivo1/" + str(patient) + "_X_val_y_val_300.pickle", 'rb') as file:

    X_val, y_val = pickle.load(file)

class H5Generator(tf.keras.utils.Sequence):

    def __init__(self, filename, x_name, y_name, batch_size):
        f = h5py.File(filename, 'r')
        self.x = f[x_name]
        self.y = f[ y_name]
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y

train_generator = H5Generator('./CHB-Definitivo1/' + str(patient) + '_X_train_y_train_300.h5', str(patient) + '_X_train',str(patient) + '_y_train',256)



model = keras.models.Sequential()


model.add(keras.layers.Conv2D(1024,5,padding='same',activation='relu', input_shape=(6, 40, 22), data_format="channels_last"))  # aggiungere data_format="channels_last"
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.MaxPooling2D())

model.add(keras.layers.Conv2D(512,5,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D( data_format="channels_last"))
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(256,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(128,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(64,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(32,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(16,3,padding='same',activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Conv2D(8,3,padding='same',activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(16,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Recall()]
)

# model.load_weights('./checkpointChb' +  str(p) + '/check_m.h5')

#ES = keras.callbacks.EarlyStopping(monitor="val_recall", mode="max", patience = 20) 
CKP = keras.callbacks.ModelCheckpoint('./checkpointChb' +  str(p) + '/check_m.h5', monitor="val_loss", mode="min")
B_CKP = keras.callbacks.ModelCheckpoint('./checkpointChb' +  str(p) + '/best_model.h5', monitor="val_recall_1", save_best_only=True, mode="max")
B_CKP2 = keras.callbacks.ModelCheckpoint('./checkpointChb' +  str(p) + '/best_loss.h5', monitor="val_loss", save_best_only=True, mode="min")
#TB = keras.callbacks.TensorBoard(log_dir='./logs' + str(patient))


batchSize = 256
model.fit(train_generator, validation_data= (X_val, y_val), validation_steps= X_val.shape[0] // batchSize, epochs = 1500, callbacks = [CKP, B_CKP, B_CKP2] )

model.save('./model_' + str(patient) + '_300.h5')


##Test

In [ ]:
import tensorflow as tf 
from tensorflow import keras
import pickle
import numpy as np
from sklearn.utils import shuffle
import os
import h5py
import math
from keras.activations import sigmoid
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from keras.metrics import FalsePositives, TruePositives, FalseNegatives, TrueNegatives
from numpy import mean, var, std
#number = "12"



#patients = ["01", "03", "05", "06", "07", "08", "09", "10", "12", "13", "14", "15", "16", "17", "18", "20", "21", "22", "23"]

            #with open("../CHB-Definitivo1/" +  str(patient) + "_X_test_y_test_300.pickle", 'rb') as file:
with open("./siena_seiz_for_fold/general/X_test_y_test.pickle", 'rb') as file:
#with open("../chb_seiz_for_fold/" +  str(patient) + "_X_test_y_test_"+ str(o) +"_fold.pickle", 'rb') as file:


        X_test, y_test = pickle.load(file)


      
      # for file in (os.listdir("../chb_seiz_for_fold/checkpoint" + str(patient) + "/fold_" + str(u))):
      #         model_path = "../chb_seiz_for_fold/checkpoint" + str(patient) + "/fold_" + str(u) +"/" + file
for file in (os.listdir("./siena_seiz_for_fold/general/checkpoint" )):
      model_path = "./siena_seiz_for_fold/general/checkpoint/" + file
      print("test - " + str(model_path))
      model = keras.models.load_model(model_path)
      print(model.metrics_names)
      score = model.evaluate(X_test, y_test, verbose=0)
      y_pred = model.predict(X_test)
      y_pred = np.where(y_pred > 0.5, 1, 0)

      # Compute the general performance
      acc = accuracy_score(y_test, y_pred)
      print("Accuracy score: ", acc)
      print(f"Test contenente {np.count_nonzero(y_test)} campioni positivi e {len(y_test)-np.count_nonzero(y_test)} campioni negativi")

      TruePos = TruePositives()
      TruePos.update_state(y_test, y_pred)
      TP = TruePos.result().numpy()
      print('True Positives: ', TP)

      TrueNeg = TrueNegatives()
      TrueNeg.update_state(y_test, y_pred)
      TN = TrueNeg.result().numpy()
      print('True Negatives: ', TN)

      FalsePos = FalsePositives()
      FalsePos.update_state(y_test, y_pred)
      FP = FalsePos.result().numpy()
      print('False Positives: ', FP)

      FalseNeg = FalseNegatives()
      FalseNeg.update_state(y_test, y_pred)
      FN = FalseNeg.result().numpy()
      print('False Negatives: ', FN)

      print('Recall: ', score[2])
      # print('Test loss:', score[0])
      spec = int(TN)/(int(TN) + int(FP))
      print('Specificity = ' + str(spec))
      
      PPV = TP/(TP+FP)
      print("Positive Predicted Value (PPV) = ", str(PPV))

      P = (np.count_nonzero(y_test))
      N = len(y_test)-np.count_nonzero(y_test)
      BPPV = (TP/P)/((TP/P) + (FP/N))
      print("Balanced Positive Predicted Value (BPPV) = ", str(BPPV))

      count_seiz = 0
      for i in range(0,len(y_test)-1):
              if (y_test[i]==1) and (y_test[i-1]==0):
      
                      count_seiz = count_seiz+1
      print(count_seiz)
      count_seiz = 0
      false_positive_list = []
      true_positive_list = []
      dev_tp = []
      dev_fp = []
      num_900 = []
      num_before_900 = []
      flag = True
      count = 0
      i = 0
      seiz = 1
      while i < len(y_test):
              if (y_test[i]==0) and (y_pred[i]==1):
                      false_positive_list.append(i)
              


              if (y_test[i]==1) and (y_test[i-1]==0):
                      conteggio_900 = 0
                      conteggio_non_900 = 0
                      #print(f"lista falsi positivi = {false_positive_list}")
                      if false_positive_list:
                              dev_fp.append(std(false_positive_list))
                      else:
                              dev_fp.append(0)
                      for e in range(0,len(false_positive_list)):
                              if false_positive_list[e] > i - 300:
                                      conteggio_900 = conteggio_900 + 1
                              else:
                                      conteggio_non_900 = conteggio_non_900 + 1
                      if false_positive_list:
                              num_900.append(((conteggio_900*100)/len(false_positive_list)))
                              num_before_900.append((conteggio_non_900*100)/len(false_positive_list))

                      else:
                              num_900.append(0)
                              num_before_900.append(0)
                      false_positive_list = []
                      j = i
                      flag = True
                      while (j < len(y_test) and (y_test[j] == 1)):
                              if (y_pred[j] == 1):
                                      true_positive_list.append(j)


                              count = count+1
                              if flag:
                                      if (y_pred[j] == 1):
                                              print(f"Seizure  {seiz} rilevata {295-count} secondi prima che l'attacco inizi")
                                              seiz = seiz +1                                
                                              count = 0
                                              flag = False
                              
                                      elif (j == len(y_test)-1) or (y_test[j]==1 and y_test[j+1]==0):
                                              print("NO trovata")
                                              seiz = seiz +1
                              j = j+1
                      i = j
                      #print(f"lista veri positivi = {true_positive_list}")
                      dev_tp.append(std(true_positive_list))
                      true_positive_list = [] 

              else:
                      count =0
                      i=i+1

      print("Deviazione Standard dei FP = ", mean(dev_fp))
      print("Deviazione Standard dei TP = ", mean(dev_tp))
      print("Numero di valori FP trovati nei 900 secondi = ", mean(num_900))
      print("Numero di valori FP trovati prima dei 900 secondi = ", mean(num_before_900))
                    

